In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies = pd.merge(movies,credits,on='title')

In [ ]:
movies.head()

In [ ]:
movies = movies[['movie_id','title', 'overview', 'genres','keywords', 'cast', 'crew']]

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
import ast
movies.iloc[0].genres

ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
def convert(obj):
  l=[]
  for i in ast.literal_eval(obj):
    l.append(i['name'])
  return l

In [ ]:
 movies['genres']=movies['genres'].apply(convert)
 movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
def convert3(obj):
  l=[]
  count=0
  for i in ast.literal_eval(obj):
    if count!=3:
      l.append(i['name'])
    else:
      break
  return l

In [ ]:
movies['cast']=movies['cast'].apply(convert)

In [ ]:
movies['crew'][0]

In [ ]:
def convert2(obj):
  l=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      l.append(i['name'])
  return l

In [ ]:
movies['crew']=movies['crew'].apply(convert)

In [ ]:
movies['overview']

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
new_df = movies[['movie_id', 'title' , 'tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" " .join(x))

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000 , stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
cv.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity

In [ ]:
def recommend(movie):
  movie_index = new_df[new_df['title'] == movie].index[0]
  distances = similarity[movie_index]
  movie_list = sorted(list(enumerate(distances)),reverse = True, key = lambda x:x[1])[1:6]

  for i in movie_list:
    print(new_df.iloc[i[0]].title)
  return 


In [ ]:
recommend('Batman Begins')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [1]:
new_df['title'].values

NameError: name 'new_df' is not defined

pickle.dump(new_df.to_dict(),open('movie-dict.pkl','wb'))

pickle.dump(similarity,open('similarity.pkl','wb'))